In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression as LR

In [2]:
root_dir = '/Users/kessapassa/OneDrive/research_log/2018_Graduate/3D/'

In [3]:
dir_list = ['people10000', 'people20000', 'people30000']
seed_list = [str(123 + i) for i in range(3)]
csv_list = ['census', 'mobile']

In [4]:
loc_list = [14, 15, 21, 22, 27, 28]

In [5]:
csv_array = {}
for _dir in dir_list:
    csv_array[_dir] = {}
    for _seed in seed_list:
        csv_array[_dir][_seed] = {}
        for _csv in csv_list:
            df = pd.read_csv(root_dir + _dir + 'seed' + _seed + '_' + _csv + '.csv',
                                                index_col=0,
                                                encoding='Shift_JISx0213')
            csv_array[_dir][_seed][_csv] = df.iloc[loc_list, :]

In [6]:
# なぜかこの時だけ21600がないので0で補間する
csv_array['people10000']['123']['census']['21600'] = 0

In [7]:
def get_ratio_per_sum(base):
    for index in range(len(base.columns)):
        base.iloc[:, index] /= base.iloc[:, index].sum()
#     base = base.round(4)
    
    return base

In [8]:
# for _dir in dir_list:
#     for _seed in seed_list:
#         for _csv in csv_list:
#             df = csv_array[_dir][_seed][_csv].copy()
#             df = get_ratio_per_sum(df)
#             df = df.fillna(0)
#             csv_array[_dir][_seed][_csv] = df

# 1時間後にどこに移動するか割合を推定する

In [9]:
times_list = [str(3600 * (i + 1)) for i in range(6)]

In [10]:
def get_model(trainX, y):
    model = LR()
    model.fit(trainX, y)
    
    return model

In [11]:
def interpolate_times(index):
    return times_list[:index+1]

In [12]:
pred = None

pred_array = {}
for _dir in dir_list:
    pred_array[_dir] = {}
    for _seed in seed_list:
        pred_array[_dir][_seed] = {}
        for _csv in csv_list:
            pred_array[_dir][_seed][_csv] = {}
            
            for index in range(len(times_list)-1):
                # seed123だけを学習し、他はテストとする
                df = csv_array[_dir][_seed][_csv].copy()
                df = get_ratio_per_sum(df)
                df = df.fillna(0)
                
                if _seed == '123':
                    model = get_model(df[interpolate_times(index)], df[times_list[index+1]])
                    pred = model.predict(df[interpolate_times(index)])
                    
                else:
                    df = df[interpolate_times(index+1)]
                    df['pred'] = pred
                    
                    columns = df.columns
                    df['diff'] = df[columns[-2]] / df[columns[-1]]
                    pred_array[_dir][_seed][_csv][times_list[index+1]] = df

In [13]:
def get_times_list(time):
    index = times_list.index(time)
    return times_list[:index+1]

In [22]:
people = 'people20000'
seed = '125'
csv = 'mobile'
time = '21600'

In [23]:
locked_times_list = get_times_list(time)
df = csv_array[people][seed][csv][locked_times_list].copy()
df['pred'] = pred_array[people][seed][csv][time]['pred'] * csv_array[people][seed][csv][time].sum()
df['diff'] = df[time] / df['pred']
df

,3600,7200,10800,14400,18000,21600,pred,diff
14,241.0,39.0,32.0,31.0,7.0,5.0,3.547315,1.409517
15,278.0,53.0,61.0,35.0,41.0,35.0,12.415601,2.819034
21,3974.0,3395.0,3137.0,3005.0,2794.0,369.0,412.375320,0.894816
22,3297.0,2608.0,2385.0,2111.0,1941.0,313.0,265.161765,1.180412
27,5185.0,4567.0,4435.0,4113.0,3786.0,412.0,484.208440,0.850873
28,3494.0,2670.0,2121.0,1816.0,1594.0,253.0,209.291560,1.208840


In [16]:
df['diff'].describe()

count    6.000000
mean     0.946418
std      0.210471
min      0.559902
25%      0.906432
50%      1.026513
75%      1.039844
max      1.154886
Name: diff, dtype: float64

In [17]:
csv_array[people][seed][csv] / csv_array[people][seed][csv].sum()

,3600,7200,10800,14400,18000,21600
14,0.014634,0.002925,0.002629,0.002790,0.000689,0.003605
15,0.016880,0.003975,0.005012,0.003150,0.004034,0.025234
21,0.241302,0.254650,0.257744,0.270453,0.274919,0.266042
22,0.200194,0.195620,0.195958,0.189992,0.190987,0.225667
27,0.314834,0.342559,0.364391,0.370174,0.372528,0.297044
28,0.212156,0.200270,0.174267,0.163442,0.156843,0.182408
